In [6]:
from google.cloud import storage, bigquery

# Set up the Google Cloud Storage client
storage_client = storage.Client()

# Set the bucket name
bucket_name = "bucket_name" # Needs to be changed
folder_prefix = "csv/"
bucket = storage_client.bucket(bucket_name)



# Set the project ID
project_id = "project_ID" # Needs to be changed

# Set up the BigQuery client with the correct project ID
bigquery_client = bigquery.Client(project=project_id)




# Set the dataset and table names
dataset_id = "dataset_ID" # Needs to be changed
table_id = "tbl_ID" # Needs to be changed

# Create the dataset if it doesn't exist
dataset_ref = bigquery_client.dataset(dataset_id)
try:
    bigquery_client.get_dataset(dataset_ref)
except:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # e.g., "US"
    dataset = bigquery_client.create_dataset(dataset)

# Create the table if it doesn't exist
table_ref = dataset_ref.table(table_id)
try:
    bigquery_client.get_table(table_ref)
except:
    table = bigquery.Table(table_ref)
    table = bigquery_client.create_table(table)


""" 
# Delete all existing records from the table
delete_query = f"DELETE FROM `{dataset_id}.{table_id}` WHERE TRUE"
bigquery_client.query(delete_query).result()

"""

# Iterate over all CSV files in the specified folder
for blob in bucket.list_blobs(prefix=folder_prefix, delimiter="/"):
    if blob.name.endswith(".csv"):
        # Load the CSV file into the BigQuery table
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            autodetect=True,
            write_disposition=bigquery.WriteDisposition.WRITE_APPEND  # This will overwrite the table data
        )

        with blob.open("rb") as source_file:
            job = bigquery_client.load_table_from_file(
                source_file,
                f"{dataset_id}.{table_id}",
                job_config=job_config,
            )

        job.result()  # Wait for the job to complete

print(f"Data from all CSV files in {bucket_name}/{folder_prefix} has been uploaded to {dataset_id}.{table_id}")

Data from all CSV files in bucket_jogos/csv/ has been uploaded to dataset_projeto.tbl_jogos
